In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
df_train = pd.read_csv('/home/users/akshay/PCPpred/data/Train.csv')
df_train

,ID,SMILES,Permeability,Sequence,MolWt
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,"['F', 'meL', 'A', 'meL', 'Ser(tBu)', 'meL', 'm...",1773.325
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,"['F', 'meF', 'P', 'L', 'Me_Phe(3-Cl)', 'T', 'L...",1745.057
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,"['Sar', 'F', 'meL', 'meL', 'F', 'meL', 'L', 'm...",1733.267
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,"['Me_Bal', 'F', 'meF', 'meL', 'T', 'meA', 'meL...",1725.281
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,"['meA', 'meL', 'meF', 'T', 'meL', 'meL', 'I', ...",1723.309
...,...,...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,"['A', 'L', 'Mono76']",402.539
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,"['A', 'L', 'H2NEt_Phe']",374.485
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,"['dL', 'S', 'Mono85']",370.494
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,"['dS', 'V', 'Mono85']",356.467


In [3]:
df_train = df_train[['ID','SMILES','Sequence','Permeability']]
df_train.loc[:,'Sequence'] = df_train['Sequence'].apply(ast.literal_eval)

In [4]:
symbol_list = []

with open('/home/users/akshay/PCPpred/data/monomer_list.txt', 'r') as file:
    for line in file:
        symbol_list.append(line.strip())
print(symbol_list)

['A', 'dA', 'meA', 'Me_dA', 'Ala(tBu)', 'Ala(indol-2-yl)', 'dAla(indol-2-yl)', 'Me_Ala(indol-2-yl)', 'Ala(5-Tet)', 'Abu', 'dAbu', 'Me_Abu', 'Me_dAbu', 'Me_Abu(morpholino)', '2Abz', 'Aib', 'Aoc(2)', '5-Ava', 'Bal', 'Me_Bal', 'HOCOCH2_Bal', 'Cys(EtO2H)_NH2', 'Cha', 'dCha', 'Me_Cha', 'D', 'meD', 'Asp_piperidide', 'Asp(OMe)', 'Asp(Ph(2-NH2))', 'dAsp(pyrrol-1-yl)', 'E', 'Glu_NH2', 'Glu(3R-Me)', 'Glu(OMe)', 'dGlu(OMe)', 'F', 'dF', 'meF', 'Me_dF', 'Phe(4-F)', 'dPhe(4-F)', 'Phe(4-CF3)', 'Phe(4-NO2)', 'Phe(CHF2)', 'dPhe(3,4-diF)', 'Et_Phe', 'H2NEt_Phe', 'Me_Phe(3-Cl)', 'Me_Phe(4-Cl)', 'Me_Phe(a,b-dehydro)', 'G', 'Bn_Gly', 'Bn(4-Cl)_Gly', 'Bn(4-OH)_Gly', 'Bu_Gly', 'iBu_Gly', 'Et_Gly', 'EtOEt_Gly', 'HOCOCH2_Gly_ol', 'MeOEt_Gly', 'NH2Bu_Gly', 'Pr_Gly', 'PhEt_Gly', 'PhPr_Gly', 'cHexCH2_Gly', 'isoamyl_Gly', 'pentyl_Gly', '3-pyridylethyl_Gly', '2-pyridylmethyl_Gly', 'd(N->O)Gly(allyl)', 'GABA', 'H', 'Hph', 'Me_Hph', 'bHph', 'Hph(2-Cl)', 'Hph(3-Cl)', 'Hph(4-Cl)', 'Hse(Et)', 'dHyp', 'Hyp(Et)', 'I', 'dI

In [5]:
print(len(symbol_list))

385


In [6]:
def calculate_monomer_composition(df, symbol_list):
    composition_data = []

    for index, row in df.iterrows():
        # print(row['Sequence'])
        composition = {symbol: 0 for symbol in symbol_list}

        for s in row['Sequence']:
            if s in composition:
                # print(s)
                composition[s] += 1

        total_length = len(row['Sequence'])
        # print(total_length)

        frequency = {symbol: count / total_length for symbol, count in composition.items()}

        composition_data.append({
            'ID': row['ID'],
            'SMILES': row['SMILES'],
            'Permeability': row['Permeability'],
            **frequency 
        })

    composition_df = pd.DataFrame(composition_data)

    return composition_df

In [7]:
df_train_mc = calculate_monomer_composition(df_train, symbol_list)
df_train_mc.shape

(5568, 388)

In [8]:
df_train_mc

,ID,SMILES,Permeability,A,dA,meA,Me_dA,Ala(tBu),Ala(indol-2-yl),dAla(indol-2-yl),...,Mono118,Mono119,Mono120,Mono121,Mono122,Mono123,Mono124,Mono125,Mono126,Mono127
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,0.066667,0.0,0.066667,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,0.000000,0.0,0.071429,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,0.071429,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,0.000000,0.0,0.066667,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,0.066667,0.0,0.066667,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,0.333333,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,0.333333,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
df_train_mc.to_csv("/home/users/akshay/PCPpred/Monomer_features/Train_mon_comp.csv", index=False)

In [12]:
df_test = pd.read_csv('/home/users/akshay/PCPpred/data/Test.csv')

df_test = df_test[['ID','SMILES','Sequence','Permeability']]
df_test.loc[:,'Sequence'] = df_test['Sequence'].apply(ast.literal_eval)

df_test_mc = calculate_monomer_composition(df_test, symbol_list)
df_test_mc.shape

df_test_mc.to_csv("/home/users/akshay/PCPpred/Monomer_features/Test_mon_comp.csv", index=False)
df_test_mc

,ID,SMILES,Permeability,A,dA,meA,Me_dA,Ala(tBu),Ala(indol-2-yl),dAla(indol-2-yl),...,Mono118,Mono119,Mono120,Mono121,Mono122,Mono123,Mono124,Mono125,Mono126,Mono127
0,908,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-7.00,0.000000,0.0,0.133333,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,923,CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](...,-7.00,0.000000,0.0,0.133333,0.066667,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,897,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.00,0.000000,0.0,0.133333,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,587,CC(C)C[C@@H]1NC(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@...,-6.74,0.071429,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,921,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.54,0.000000,0.0,0.133333,0.066667,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0.333333,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1388,2485,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@@H](Cc2cccc...,-4.80,0.333333,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1389,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0.250000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1390,2513,C[C@H]1NCCCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[...,-7.80,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
#Amino acid composition 20 natural and X as non natural amino acids

In [14]:
mono = pd.read_csv('/home/users/akshay/PCPpred/data/CycPeptMPDB_Monomer_All (2).csv')
mono = mono[['Symbol','Natural_Analog']]
mono_dict = mono.set_index('Symbol')['Natural_Analog'].to_dict()
mono_dict

{'A': 'A',
 'dA': 'A',
 'meA': 'A',
 'Me_dA': 'A',
 'Ala(tBu)': 'A',
 'Ala(indol-2-yl)': 'A',
 'dAla(indol-2-yl)': 'A',
 'Me_Ala(indol-2-yl)': 'A',
 'Ala(5-Tet)': 'A',
 'Abu': 'X',
 'dAbu': 'X',
 'Me_Abu': 'X',
 'Me_dAbu': 'X',
 'Me_Abu(morpholino)': 'X',
 '2Abz': 'X',
 'Aib': 'X',
 'Aoc(2)': 'L',
 '5-Ava': 'X',
 'Bal': 'A',
 'Me_Bal': 'A',
 'HOCOCH2_Bal': 'A',
 'Cys(EtO2H)_NH2': 'C',
 'Cha': 'A',
 'dCha': 'A',
 'Me_Cha': 'A',
 'D': 'D',
 'meD': 'D',
 'Asp_piperidide': 'D',
 'Asp(OMe)': 'D',
 'Asp(Ph(2-NH2))': 'D',
 'dAsp(pyrrol-1-yl)': 'D',
 'E': 'E',
 'Glu_NH2': 'E',
 'Glu(3R-Me)': 'E',
 'Glu(OMe)': 'E',
 'dGlu(OMe)': 'E',
 'F': 'F',
 'dF': 'F',
 'meF': 'F',
 'Me_dF': 'F',
 'Phe(4-F)': 'F',
 'dPhe(4-F)': 'F',
 'Phe(4-CF3)': 'F',
 'Phe(4-NO2)': 'F',
 'Phe(CHF2)': 'F',
 'dPhe(3,4-diF)': 'F',
 'Et_Phe': 'F',
 'H2NEt_Phe': 'F',
 'Me_Phe(3-Cl)': 'F',
 'Me_Phe(4-Cl)': 'F',
 'Me_Phe(a,b-dehydro)': 'F',
 'G': 'G',
 'Bn_Gly': 'G',
 'Bn(4-Cl)_Gly': 'G',
 'Bn(4-OH)_Gly': 'G',
 'Bu_Gly': 'G',
 '

In [15]:
amino_acid_list =  list(set(mono['Natural_Analog'].to_list()))
amino_acid_list

['W',
 'H',
 'M',
 'L',
 'X',
 'A',
 'E',
 'Q',
 'T',
 'N',
 'D',
 'V',
 'Y',
 'P',
 'S',
 'G',
 'C',
 'R',
 'F',
 'I',
 'K']

In [16]:
def seq2seq(sequence):
    seq = ''
    for s in sequence:
        if s in mono_dict.keys():
            seq += mono_dict[s]
    return seq

In [17]:
def calculate_aac(df, symbol_list):
    composition_data = []

    for index, row in df.iterrows():
        # print(row['Sequence'])
        composition = {symbol: 0 for symbol in symbol_list}

        for s in row['Natural_analog_sequence']:
            if s in composition:
                # print(s)
                composition[s] += 1

        total_length = len(row['Natural_analog_sequence'])
        # print(total_length)

        frequency = {symbol: count / total_length for symbol, count in composition.items()}

        composition_data.append({
            'ID': row['ID'],
            'SMILES': row['SMILES'],
            'Permeability': row['Permeability'],
            **frequency 
        })

    composition_df = pd.DataFrame(composition_data)

    return composition_df

In [18]:
df_train = pd.read_csv("/home/users/akshay/PCPpred/data/Train.csv")
df_train.loc[:,'Sequence'] = df_train['Sequence'].apply(ast.literal_eval)
df_train.loc[:, 'Natural_analog_sequence'] = df_train['Sequence'].apply(lambda x :seq2seq(x))
df_train = calculate_aac(df_train, amino_acid_list)
print('Shape',df_train.shape)
df_train.to_csv('/home/users/akshay/PCPpred/Monomer_features/Train_aac.csv',index=False)
df_train

Shape (5568, 24)


,ID,SMILES,Permeability,W,H,M,L,X,A,E,...,V,Y,P,S,G,C,R,F,I,K
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,0.0,0.0,0.0,0.266667,0.066667,0.133333,0.0,...,0.000000,0.0,0.000000,0.066667,0.000000,0.0,0.000000,0.200000,0.133333,0.0
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,0.0,0.0,0.0,0.285714,0.071429,0.071429,0.0,...,0.000000,0.0,0.071429,0.000000,0.000000,0.0,0.000000,0.285714,0.071429,0.0
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,0.0,0.0,0.0,0.357143,0.071429,0.071429,0.0,...,0.000000,0.0,0.000000,0.000000,0.071429,0.0,0.071429,0.285714,0.000000,0.0
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,0.0,0.0,0.0,0.266667,0.066667,0.133333,0.0,...,0.133333,0.0,0.000000,0.066667,0.000000,0.0,0.000000,0.133333,0.066667,0.0
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,0.0,0.0,0.0,0.333333,0.066667,0.133333,0.0,...,0.066667,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.133333,0.133333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,0.0,0.0,0.0,0.333333,0.000000,0.333333,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.333333,0.000000,0.0
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,0.0,0.0,0.0,0.333333,0.000000,0.333333,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.333333,0.000000,0.0
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,0.0,0.0,0.0,0.333333,0.000000,0.333333,0.0,...,0.000000,0.0,0.000000,0.333333,0.000000,0.0,0.000000,0.000000,0.000000,0.0
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,0.0,0.0,0.0,0.000000,0.000000,0.333333,0.0,...,0.333333,0.0,0.000000,0.333333,0.000000,0.0,0.000000,0.000000,0.000000,0.0


In [19]:
df_test = pd.read_csv("/home/users/akshay/PCPpred/data/Test.csv")
df_test.loc[:,'Sequence'] = df_test['Sequence'].apply(ast.literal_eval)
df_test.loc[:, 'Natural_analog_sequence'] = df_test['Sequence'].apply(lambda x :seq2seq(x))
df_test = calculate_aac(df_test, amino_acid_list)
print('Shape',df_test.shape)
df_test.to_csv('/home/users/akshay/PCPpred/Monomer_features/Test_aac.csv',index=False)
df_test

Shape (1392, 24)


,ID,SMILES,Permeability,W,H,M,L,X,A,E,...,V,Y,P,S,G,C,R,F,I,K
0,908,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-7.00,0.0,0.000000,0.0,0.200000,0.066667,0.133333,0.0,...,0.066667,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.200000,0.200000,0.0
1,923,CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](...,-7.00,0.0,0.000000,0.0,0.266667,0.066667,0.200000,0.0,...,0.000000,0.000000,0.000000,0.066667,0.000000,0.0,0.0,0.133333,0.133333,0.0
2,897,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.00,0.0,0.000000,0.0,0.133333,0.066667,0.133333,0.0,...,0.200000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.200000,0.133333,0.0
3,587,CC(C)C[C@@H]1NC(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@...,-6.74,0.0,0.071429,0.0,0.357143,0.071429,0.071429,0.0,...,0.000000,0.000000,0.000000,0.000000,0.071429,0.0,0.0,0.285714,0.000000,0.0
4,921,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.54,0.0,0.000000,0.0,0.333333,0.066667,0.200000,0.0,...,0.000000,0.000000,0.000000,0.066667,0.000000,0.0,0.0,0.133333,0.066667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0.0,0.000000,0.0,0.333333,0.000000,0.333333,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.333333,0.000000,0.0
1388,2485,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@@H](Cc2cccc...,-4.80,0.0,0.000000,0.0,0.333333,0.000000,0.333333,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.333333,0.000000,0.0
1389,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0.0,0.000000,0.0,0.000000,0.250000,0.250000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.250000,0.0,0.0,0.250000,0.000000,0.0
1390,2513,C[C@H]1NCCCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[...,-7.80,0.0,0.000000,0.0,0.000000,0.000000,0.333333,0.0,...,0.000000,0.333333,0.333333,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
